In [4]:
import pandas as pd
import os

base_path = '../data'
arquivo_pix = rf'{base_path}\Estatisticas de transações Pix.csv'
arquivo_pagamentos = rf'{base_path}\Meios de Pagamentos Mensais.csv'
arquivo_ipca = rf'{base_path}\ipca.csv'
arquivo_selic = rf'{base_path}\selic.csv'


def carregar_e_preparar(nome_arquivo, coluna_data, sep=',', formato_data=None, encoding='utf-8'):
    """
    Carrega um CSV, converte a coluna de data e a define como índice.
    """
    try:
        df = pd.read_csv(nome_arquivo, sep=sep, decimal=',', thousands='.', encoding=encoding)
    except UnicodeDecodeError:
        print(f"AVISO: Falha no encoding 'utf-8'. Tentando 'latin1' para {nome_arquivo}...")
        df = pd.read_csv(nome_arquivo, sep=sep, decimal=',', thousands='.', encoding='latin1')
    except FileNotFoundError:
        print(f"ERRO: Arquivo '{nome_arquivo}' não encontrado.")
        return None
    except Exception as e:
        print(f"Ocorreu um erro inesperado ao ler o arquivo {nome_arquivo}: {e}")
        return None

    try:
        df[coluna_data] = pd.to_datetime(df[coluna_data], format=formato_data, errors='coerce')
        df = df.dropna(subset=[coluna_data])
        df = df.set_index(coluna_data)
        df = df.sort_index()
        print(f"Arquivo '{nome_arquivo}' carregado com sucesso.")
        return df

    except KeyError:
        print(f"ERRO: A coluna de data '{coluna_data}' não foi encontrada em '{nome_arquivo}'.")
        print(f"Colunas disponíveis: {df.columns.to_list()}")
    except Exception as e:
        print(f"Ocorreu um erro ao processar as datas em {nome_arquivo}: {e}")
    return None

df_pix = carregar_e_preparar(arquivo_pix, coluna_data='AnoMes', sep=',', formato_data='%Y%m')

df_pagamentos = carregar_e_preparar(arquivo_pagamentos, coluna_data='AnoMes', sep=',', formato_data='%Y%m')

df_selic = carregar_e_preparar(arquivo_selic, coluna_data='data', sep=';', formato_data='%d/%m/%Y')

df_ipca = carregar_e_preparar(arquivo_ipca, coluna_data='data', sep=';', formato_data='%d/%m/%Y')

if df_selic is not None:
    try:
        df_selic = df_selic.rename(columns={'valor': 'taxa_selic'})
        print("Coluna 'valor' do SELIC renomeada para 'taxa_selic'")
    except KeyError:
        print("ERRO: Coluna 'valor' não encontrada em 'df_selic'.")

if df_ipca is not None:
    try:
        df_ipca = df_ipca.rename(columns={'valor': 'indice_ipca'})
        print("Coluna 'valor' do IPCA renomeada para 'indice_ipca'")
    except KeyError:
        print("ERRO: Coluna 'valor' não encontrada em 'df_ipca'.")


df_selic_mensal = None
if df_selic is not None:
    try:
        df_selic_mensal = df_selic[['taxa_selic']].resample('MS').mean()

        df_selic_mensal['taxa_selic'] = df_selic_mensal['taxa_selic'].replace(0, pd.NA)

        df_selic_mensal = df_selic_mensal.ffill()

        print("Dados da SELIC reamostrados e limpos com sucesso.")
    except Exception as e:
        print(f"Erro ao reamostrar SELIC: {e}")

print("\n--- Verificando duplicatas nos índices (datas) ---")
dataframes_para_checar = {
    "df_pix": df_pix, "df_pagamentos": df_pagamentos,
    "df_selic_mensal": df_selic_mensal, "df_ipca": df_ipca
}
dataframes_corrigidos = {}

for nome_df, df in dataframes_para_checar.items():
    if df is not None:
        if not df.index.is_unique:
            print(f"AVISO: {nome_df} tem datas duplicadas! Removendo duplicatas (mantendo a primeira)...")
            df_corrigido = df[~df.index.duplicated(keep='first')]
            dataframes_corrigidos[nome_df] = df_corrigido
        else:
            print(f"{nome_df} está OK (índice único).")
            dataframes_corrigidos[nome_df] = df
    else:
        print(f"{nome_df} não foi carregado, pulando verificação.")
print("Verificação de duplicatas concluída.")

print("\n--- Iniciando junção (merge) dos DataFrames ---")
df_pix_limpo = dataframes_corrigidos.get("df_pix")
df_pag_limpo = dataframes_corrigidos.get("df_pagamentos")
df_selic_limpo = dataframes_corrigidos.get("df_selic_mensal")
df_ipca_limpo = dataframes_corrigidos.get("df_ipca")

dataframes_para_unir = []

if df_pix_limpo is not None:
    try:
        dataframes_para_unir.append(df_pix_limpo[['QUANTIDADE', 'VALOR']])
    except KeyError:
        print("ERRO: Colunas 'Quantidade' ou 'Valor' não encontradas em 'df_pix'.")
        print(f"Colunas disponíveis no df_pix: {df_pix_limpo.columns.to_list()}")

if df_selic_limpo is not None:
    if 'taxa_selic' in df_selic_limpo.columns:
        dataframes_para_unir.append(df_selic_limpo[['taxa_selic']])

if df_ipca_limpo is not None:
    if 'indice_ipca' in df_ipca_limpo.columns:
        dataframes_para_unir.append(df_ipca_limpo[['indice_ipca']])

if df_pag_limpo is not None:
    try:
        colunas_pagamentos = ['quantidadeTED', 'valorTED', 'quantidadeBoleto', 'valorBoleto',
                            'quantidadeDOC', 'valorDOC', 'quantidadeCheque', 'valorCheque',
                            'quantidadeTEC', 'valorTEC', 'quantidadePix', 'valorPix']
        colunas_existentes = [col for col in colunas_pagamentos if col in df_pag_limpo.columns]
        dataframes_para_unir.append(df_pag_limpo[colunas_existentes])
    except KeyError:
        print(f"ERRO: Colunas de 'df_pagamentos' não encontradas.")

if len(dataframes_para_unir) >= 2:
    df_final = pd.concat(dataframes_para_unir, axis=1, join='inner')

    if df_final.empty:
        print("AVISO: 'join' interno resultou em DataFrame vazio. As datas dos arquivos ainda não estão alinhadas.")
        print("Rodando diagnóstico de datas...")
        for nome, df in dataframes_corrigidos.items():
            if df is not None and not df.empty:
                print(f"Período de {nome}: {df.index.min()} até {df.index.max()}")
            elif df is None:
                print(f"Diagnóstico: {nome} não foi carregado (None).")
            else:
                print(f"Diagnóstico: {nome} está vazio.")


    print("\n--- DataFrame Final Combinado (Head) ---")
    print(df_final.head())

    print("\n--- Verificando Dados Faltantes no DataFrame Final ---")
    print(df_final.isnull().sum())

    caminho_saida = rf'{base_path}\dados_consolidados_para_analise.csv'
    df_final.to_csv(caminho_saida)
    print(f"\nDataFrame final salvo em: {caminho_saida}")
else:
    print("\nNão foi possível gerar o DataFrame final. Menos de 2 DataFrames foram carregados com sucesso.")

Arquivo '../data\Estatisticas de transações Pix.csv' carregado com sucesso.
Arquivo '../data\Meios de Pagamentos Mensais.csv' carregado com sucesso.
Arquivo '../data\selic.csv' carregado com sucesso.
Arquivo '../data\ipca.csv' carregado com sucesso.
Coluna 'valor' do SELIC renomeada para 'taxa_selic'
Coluna 'valor' do IPCA renomeada para 'indice_ipca'
Dados da SELIC reamostrados e limpos com sucesso.

--- Verificando duplicatas nos índices (datas) ---
AVISO: df_pix tem datas duplicadas! Removendo duplicatas (mantendo a primeira)...
df_pagamentos está OK (índice único).
df_selic_mensal está OK (índice único).
df_ipca está OK (índice único).
Verificação de duplicatas concluída.

--- Iniciando junção (merge) dos DataFrames ---

--- DataFrame Final Combinado (Head) ---
            QUANTIDADE        VALOR  taxa_selic  indice_ipca  quantidadeTED  \
2020-11-01         327    199116.46    0.007469         0.41      191703.38   
2020-12-01           1      1512.00    0.007469         2.04      